In [60]:
import tensorflow as tf

import numpy as np

In [2]:
tf.__version__

'1.3.0'

In [3]:
dialogues_path = "./data/movie_lines.txt"

### Read data

In [21]:
SOS_TOKEN = "<s>"
EOS_TOKEN = "</s>"

In [22]:
dialogue_lines = list()
with open(dialogues_path) as dialogues_file:
    for line in dialogues_file:
        line = line.strip().lower()
        split_line = line.split(' +++$+++ ')
        dialogue_lines.append(SOS_TOKEN + " " + split_line[4] + " " + EOS_TOKEN)

In [23]:
dialogue_lines[:10]

['<s> they do not! </s>',
 '<s> they do to! </s>',
 '<s> i hope so. </s>',
 '<s> she okay? </s>',
 "<s> let's go. </s>",
 '<s> wow </s>',
 "<s> okay -- you're gonna need to learn how to lie. </s>",
 '<s> no </s>',
 '<s> i\'m kidding.  you know how sometimes you just become this "persona"?  and you don\'t know how to quit? </s>',
 '<s> like my fear of wearing pastels? </s>']

### Get language stats

In [40]:
from nltk.tokenize import word_tokenize

In [113]:
def text_to_sequences(dialogue_lines):
    
    word_to_index_map = {SOS_TOKEN: 1, EOS_TOKEN: 2}
    index_to_word_map = {1: SOS_TOKEN, 2: EOS_TOKEN}
    index = 3
    
    dialogue_sequences = list()
    
    for line in dialogue_lines:
        dialogue_sequence = list()
        tokens = word_tokenize(line)
        for token in tokens:
            if token in word_to_index_map:
                dialogue_sequence.append(word_to_index_map[token])
            else:
                word_to_index_map[token] = index
                index_to_word_map[index] = token
                dialogue_sequence.append(index)
                index += 1
        dialogue_sequences.append(dialogue_sequence)
                
    return dialogue_sequences, word_to_index_map, index_to_word_map

In [114]:
sequences, word_to_index_map, index_to_word_map = text_to_sequences(dialogue_lines)

In [116]:
len(sequences)

100

In [ ]:
tensor_sequences = tf.convert_to_tensor(sequences)

In [46]:
# word_to_index_map, index_to_word_map

In [74]:
VOCAB_SIZE = len(word_to_index_map)
EMBEDDING_SIZE = 50
LSTM_SIZE = 100

In [75]:
VOCAB_SIZE

375

In [90]:
from tensorflow.contrib.rnn import LSTMCell

In [111]:
tf.reset_default_graph()

In [ ]:
class Seq2SeqModel(object):
    